credit to https://www.freecodecamp.org/news/building-a-neural-network-from-scratch/
for the code

In [10]:
import numpy as np
import matplotlib.pyplot as plt

In [11]:
def init_params(layer_dims):
    np.random.seed(3)
    params = {}
    L = len(layer_dims)

    for l in range(1, L):
        params["W" + str(l)] = np.random.randn(layer_dims[1], layer_dims[l - 1]) * 0.01
        params["b" + str(l)] = np.zeroes(layer_dims[1], 1)

In [12]:
# Z = w*X + b
# W = weights, b = bias, x = input


def sigmoid(Z):
    A = 1 / (1 + np.exp(np.dot(-1, Z)))
    cache = Z
    return A, cache

In [13]:
def forward_prop(X, params):
    A = X  # input to first layer i.e. training data
    caches = []
    L = len(params) / 2
    for l in range(1, L + 1):
        A_prev = A
        # Linear hypothesis
        Z = np.dot(params["W" + str(l)], A_prev) + params["b" + str(l)]
        # Storing linear cache
        linear_cache = (A_prev, params["W" + str(l)], params["b" + str(l)])
        # Applying sigmoid on linear hypothesis
        A, activation_cache = sigmoid(Z)

        # Storing both linear and activation cache / previous
        # and current cache

        caches.append((linear_cache, activation_cache))
    return A, caches